In [31]:
import requests
from bs4 import BeautifulSoup

In [73]:
url = u'https://twitter.com/search?q=matka%20since%3A2017-12-01%20until%3A2017-12-31&l=pl&src=typd&f=tweets'

In [13]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [74]:
browser = webdriver.Chrome()
browser.get(url)
time.sleep(1)
body = browser.find_element_by_tag_name('body')
for _ in range(15):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)

In [75]:
tweets = [tweet.text for tweet in browser.find_elements_by_class_name("tweet-text") if 'matka' in tweet.text]
len(tweets)

18

In [76]:
for tweet in tweets:
    print(tweet)

Jeżeli matka chodzi sprawdzać co 5 min czy pale to powodzenia
matka mi wlazła do pokoju i że mam gasić więc tylko wybuchłem śmiechem. ile ja mam kurwa lat przepraszam bardzoXD
matka, córka seks Baran Koziorożec seks 
w sumie miłość matki do dziecka potrafi być większa a może nie tyle co większa co matka potrafi się poświęcić 
Na tym polega manipulacja zeby komus przylozyc. Ta pani pisze, ze jest matka. Zastanawiajace, ze moze uczyc swoje dzieci klamstw i manipulacji.
Widzę pierwszą od dawna dramę poza Azjatyckim rp, czuję się jak dumna matka, że to tym razem nie my.
Matka Anny Przybylskiej szokuje. Ujawniła okropnie smutny fakt o córce http://pikio.pl/matka-anny-przybylskiej-szokuje-ujawnila-okropnie-smutny-fakt-o-corce/ … przez @PikioPL
Nie słyszałeś? Jego matka do mnie dzwoniła i przekazała nieprzyjemne wieści. Został postrzelony, a kula nieszczęśliwie przebiła tętnice glow
w wakacje moja matka jedzie z bratem na kolonie więc przez 2 tygodnie mam chatę wolną od rana do 17
tylko teraz